In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("/home/missinguser/CSE/single-stage-opt/hybrid_tokamak/laptop/")
import latexplot
latexplot.set_cmap() 
from spec_rename import SpecRename

from simsopt import mhd
from simsopt import geo
from simsopt import objectives
import matplotlib.pyplot as plt

import glob

directories = {
  "QA_30": "../../freeb_high_aspect_combined",
  # "QA_20": "../../freeb_12-11-16-48-29",
  "QA_20": "../../freeb_01-02-01-27-47",
  "QH_failed": "../../freeb_12-23-01-03-52",
  "QA_20_fixb": "../../fixb_12-31-16-42-24",
               }

In [2]:
def plot_booz_nofig(vmec, **kwargs):
    import booz_xform
    from matplotlib import cm
    vmec.run()
    sarr = [1.0]
    print("aspect", vmec.boundary.aspect_ratio())
    mpol = kwargs.pop("mpol", 32)
    ntor = kwargs.pop("ntor", 32)
    boozer = mhd.Boozer(vmec, mpol, ntor)
    boozer.register(sarr)
    boozer.run()

    if "cmap" not in kwargs:
        kwargs["cmap"] = cm.plasma
    if "fill" not in kwargs:
        kwargs["fill"] = False

    booz_xform.surfplot(boozer.bx, 0, **kwargs)


def plot_booz_spec(filename, label=None):
    if "input." in filename: 
        vmec = mhd.Vmec(filename, verbose=False)
        specf = filename.replace("input.", "")
    else:
        vmec = mhd.Vmec(verbose=False)
        vmec.indata.phiedge = 8e-3
        with SpecRename(filename) as specf:
            print(f"renamed {filename} to {specf}")
            spec = mhd.Spec(specf, tolerance=1e-10, verbose=False)
            vmec.boundary = spec.boundary.copy()
            # vmec.boundary = geo.SurfaceRZFourier.from_vmec_input(specf)
    plot_booz_nofig(vmec, ncontours=16)
    label = label or filename
    plt.title(label)

In [3]:

import re
import os
def find_matching_end(directory):
    # Regular expression to match the desired file pattern
    pattern = re.compile(r'.*_000_\d{6}\.sp\.end$')

    # Get all files in the directory matching the pattern
    matching_files = [os.path.join(directory, f) for f in os.listdir(directory) if pattern.match(f)]
    return sorted(matching_files)


def find_matching_input(directory):
    # Regular expression to match the desired file pattern
    pattern = re.compile(r'input\..*_000_\d{6}$')

    # Get all files in the directory matching the pattern
    matching_files = [os.path.join(directory, f) for f in os.listdir(directory) if pattern.match(f)]
    return sorted(matching_files)

def dof_from_mpol(mpol):
    return mpol*(mpol*2+1)+mpol 


In [4]:

for key, directory in directories.items():
    subdirs = glob.glob(f"{directory}/mpol*")
    subdirs = sorted(subdirs)

    cols = 2
    nrows = int(np.ceil((len(subdirs)+1)/cols))
    fig, axs = plt.subplots(nrows, cols, figsize=latexplot.get_size(1, (nrows, cols)), sharex=True, sharey=True)    
    axs = axs.flatten()
    subplot = 0
    plt.sca(axs[subplot])

    if "freeb" in directory:
        # Freeboundary
        first_file = find_matching_end(subdirs[0])[0]
        print(first_file)
        plot_booz_spec(first_file, f"$M=N=0$, DOFs=0")
    else:
        first_file = find_matching_input(subdirs[0])[0]
        print(first_file)
        plot_booz_spec(first_file, f"$M=N=0$, DOFs=0")
    plt.xlabel("")
    subplot += 1
    for i, subdir in enumerate(subdirs):
        plt.sca(axs[subplot])
        res = int(subdir.split("mpol")[-1])
        if "freeb" in directory:
            latest_file = find_matching_end(subdir)[-1]
            plot_booz_spec(latest_file, f"$M=N={res}$, DOFs={dof_from_mpol(res)}")
        else:
            # Fixboundary
            latest_file = find_matching_input(subdir)[-1]
            plot_booz_spec(latest_file, f"$M=N={res}$, DOFs={2*dof_from_mpol(res)}")

        if (i+1) <= nrows*(cols-1):
            plt.xlabel("")
        if (i+1) % cols > 0:
            plt.ylabel("")
        plt.gca().label_outer()
        subplot += 1
    pbasename = directory+"/"+key+"_booz_forms"
    latexplot.savenshow(pbasename)
    for filename in glob.glob(f"{pbasename}*"):
        import subprocess
        subprocess.check_call(["mv", filename, "./"])
        

../../freeb_high_aspect_combined/mpol1/rotating_ellipse_fb_low_000_000000.sp.end
renamed ../../freeb_high_aspect_combined/mpol1/rotating_ellipse_fb_low_000_000000.sp.end to ../../freeb_high_aspect_combined/mpol1/rotating_ellipse_fb_low_000_000000.sp
aspect 29.50917787893153
renamed ../../freeb_high_aspect_combined/mpol1/rotating_ellipse_fb_low_000_000090.sp.end to ../../freeb_high_aspect_combined/mpol1/rotating_ellipse_fb_low_000_000090.sp
aspect 29.24780539526665
renamed ../../freeb_high_aspect_combined/mpol2/rotating_ellipse_fb_low_000_000103.sp.end to ../../freeb_high_aspect_combined/mpol2/rotating_ellipse_fb_low_000_000103.sp
aspect 28.960727398244124
renamed ../../freeb_high_aspect_combined/mpol3/rotating_ellipse_fb_low_000_000027.sp.end to ../../freeb_high_aspect_combined/mpol3/rotating_ellipse_fb_low_000_000027.sp
aspect 29.159203685688016
renamed ../../freeb_high_aspect_combined/mpol4/rotating_ellipse_fb_low_000_000164.sp.end to ../../freeb_high_aspect_combined/mpol4/rotating_e

/home/missinguser/CSE/single-stage-opt/hybrid_tokamak/laptop/latexplot.py:91: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


../../freeb_01-02-01-27-47/mpol1/rotating_ellipse_fb_low_000_000000.sp.end
renamed ../../freeb_01-02-01-27-47/mpol1/rotating_ellipse_fb_low_000_000000.sp.end to ../../freeb_01-02-01-27-47/mpol1/rotating_ellipse_fb_low_000_000000.sp
aspect 19.77271270240201
renamed ../../freeb_01-02-01-27-47/mpol1/rotating_ellipse_fb_low_000_000067.sp.end to ../../freeb_01-02-01-27-47/mpol1/rotating_ellipse_fb_low_000_000067.sp
aspect 19.278196901570688
renamed ../../freeb_01-02-01-27-47/mpol2/rotating_ellipse_fb_low_000_000222.sp.end to ../../freeb_01-02-01-27-47/mpol2/rotating_ellipse_fb_low_000_000222.sp
aspect 19.79654515976074
renamed ../../freeb_01-02-01-27-47/mpol3/rotating_ellipse_fb_low_000_000275.sp.end to ../../freeb_01-02-01-27-47/mpol3/rotating_ellipse_fb_low_000_000275.sp
aspect 19.840249014883135
renamed ../../freeb_01-02-01-27-47/mpol4/rotating_ellipse_fb_low_000_000398.sp.end to ../../freeb_01-02-01-27-47/mpol4/rotating_ellipse_fb_low_000_000398.sp
aspect 19.817238089568875
renamed ../.

/home/missinguser/CSE/single-stage-opt/hybrid_tokamak/laptop/latexplot.py:91: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


../../freeb_12-23-01-03-52/mpol1/rotating_ellipse_fb_low_000_000000.sp.end
renamed ../../freeb_12-23-01-03-52/mpol1/rotating_ellipse_fb_low_000_000000.sp.end to ../../freeb_12-23-01-03-52/mpol1/rotating_ellipse_fb_low_000_000000.sp
aspect 10.12419232880214
renamed ../../freeb_12-23-01-03-52/mpol1/rotating_ellipse_fb_low_000_000082.sp.end to ../../freeb_12-23-01-03-52/mpol1/rotating_ellipse_fb_low_000_000082.sp
aspect 10.095336003355145
renamed ../../freeb_12-23-01-03-52/mpol2/rotating_ellipse_fb_low_000_000274.sp.end to ../../freeb_12-23-01-03-52/mpol2/rotating_ellipse_fb_low_000_000274.sp
aspect 10.023484655366145
renamed ../../freeb_12-23-01-03-52/mpol3/rotating_ellipse_fb_low_000_000696_000_000000.sp.end to ../../freeb_12-23-01-03-52/mpol3/rotating_ellipse_fb_low_000_000696_000_000000.sp
aspect 10.034397045177046
renamed ../../freeb_12-23-01-03-52/mpol4/rotating_ellipse_fb_low_000_000859.sp.end to ../../freeb_12-23-01-03-52/mpol4/rotating_ellipse_fb_low_000_000859.sp
aspect 10.03457

/home/missinguser/CSE/single-stage-opt/hybrid_tokamak/laptop/latexplot.py:91: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


../../fixb_12-31-16-42-24/mpol1/input.rot_ellipse_000_000000
aspect 19.77258262965083
aspect 19.43156746298171
aspect 19.59578885672908
aspect 19.59579683666826
aspect 19.595783728213213
aspect 19.595784875226006


/home/missinguser/CSE/single-stage-opt/hybrid_tokamak/laptop/latexplot.py:91: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
